Integrantes:
- Sergio Escudero Tabares - 2040801
- Luisa María Cárdenas Lopez - 1823494
- Stephania Noguera Romero - 2125854

In [2]:
!pip install datasets
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.5 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset
import nltk

nltk.download('stopwords')

all_wiki = load_dataset('large_spanish_corpus', name='DGT',split="train")
all_wiki

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for large_spanish_corpus contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/large_spanish_corpus
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Pass

Generating train split:   0%|          | 0/3168368 [00:00<?, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 3168368
})

##Limpieza y preparación

In [4]:
sentences = []

for i, sent in enumerate( all_wiki['text']):
    if i >= 0:
        if len(sent) > 1:
            sente = sent.split()
            sentences.append(sente)
           # tokens = word_tokenize(sentences)

print('No. Oraciones en train dataset: ', len(sentences))

No. Oraciones en train dataset:  3168368


In [5]:
print(sentences[0])

['Reglamento', 'de', 'Ejecución', '(UE)', 'no', '1073/2012', 'de', 'la', 'Comisión']


In [6]:
from nltk.corpus import stopwords
from tqdm.auto import tqdm
import re, string

sentences.pop(0)
sentencias1=[]

for nom in tqdm(sentences):
    newstring = [i for i in nom if not i.isdigit()]
    newstring = [re.sub(r'[0-9]','', w) for w in newstring]
    stop_words = stopwords.words('spanish')
    newstring = [w for w in newstring if not w in stop_words]
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    newstring = [re_punc.sub('', w) for w in newstring]
    newstring = [re.sub("\!|\'|\?|\¿|\¡|\«","",w) for w in newstring]
    sentencias1.append(newstring)
    #print(newstring)
    #print(len(newstring))

  0%|          | 0/3168367 [00:00<?, ?it/s]

In [7]:

from tqdm.auto import tqdm  # for our loading bar
sentences = []
sentences_dirty = []
diccionario = dict()

num_lines2 = len(sentencias1)

for i, sent in tqdm(enumerate(sentencias1), total=num_lines2):
    sente = []
    for k, word in enumerate(sent):
        sente.append(word)
        if word not in diccionario:
            diccionario[word] = 1
        else:
            diccionario[word] += 1

    sentences.append(sente)
    sentences_dirty.append(sente)

print(sentences[1:5])

  0%|          | 0/3168367 [00:00<?, ?it/s]

[['establecen', 'valores', 'importación', 'alzado', 'determinación', 'precio', 'entrada', 'determinadas', 'frutas', 'hortalizas'], ['LA', 'COMISIÓN', 'EUROPEA'], ['Visto', 'Tratado', 'Funcionamiento', 'Unión', 'Europea'], ['Visto', 'Reglamento', 'CE', '', 'Consejo', 'octubre', '', 'crea', 'organización', 'común', 'mercados', 'agrícolas', 'establecen', 'disposiciones', 'específicas', 'determinados', 'productos', 'agrícolas', 'Reglamento', 'único', 'OCM', '']]


#1) La construcción de los modelos de FastTex y Word2vec

##Word2vec

In [8]:
!pip install gensim

In [2]:
from gensim.models import FastText
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

dataset = load_dataset('large_spanish_corpus', name='DGT')

similar_words = ["Importación", "Europea", "permita", "Madrid", "Bogotá"]

# 300 words
modelW2V300 = FastText(vector_size=300, window=5, min_count=1, workers=4)
modelW2V300.build_vocab(corpus_iterable=sentences[1:300])

for word in similar_words:
  print(word)
  try:
    print(modelW2V300.wv.most_similar(word))
  except:
    print(word + " not in vocabulary")

NameError: name 'load_dataset' is not defined

In [10]:
# 1M words
modelW2V1M = FastText(vector_size=300, window=5, min_count=1, workers=4)
modelW2V1M.build_vocab(corpus_iterable=sentences[1:1000000])

for word in similar_words:
  print(word)
  try:
    print(modelW2V1M.wv.most_similar(word))
  except:
    print(word + " not in vocabulary")

Importación
[('Importación–Exportación', 0.8497372269630432), ('ImportaciónExportación', 0.8420952558517456), ('Importación»', 0.8189637064933777), ('ExportaciónImportación', 0.8016979098320007), ('importación', 0.7796217799186707), ('SíNoImportación', 0.7567048072814941), ('“importación', 0.7450816035270691), ('exportaciónimportación', 0.7324195504188538), ('reimportación', 0.7286826372146606), ('aportación', 0.7119565606117249)]
Europea
[('Europeaj', 0.7743606567382812), ('Europea»', 0.757809579372406), ('Europea”', 0.7416638135910034), ('Europeas', 0.7369647026062012), ('Europea“', 0.7276790142059326), ('European', 0.7201011776924133), ('Europeaa', 0.717343270778656), ('Europe', 0.7121480703353882), ('Europeas–', 0.7092788219451904), ('Europeana', 0.6981995701789856)]
permita
[('permitan', 0.714332640171051), ('permite', 0.6271286606788635), ('permiten', 0.5809476375579834), ('permito', 0.5613845586776733), ('permitirá', 0.5593917965888977), ('permitir', 0.5448753833770752), ('permi

In [1]:
# all words
modelW2VAll = FastText(vector_size=300, window=5, min_count=1, workers=4)
modelW2VAll.build_vocab(corpus_iterable=sentences)

for word in similar_words:
  print(word)
  try:
    print(modelW2VAll.wv.most_similar(word))
  except:
    print(word + " not in vocabulary")

NameError: name 'FastText' is not defined

##FastTex


In [ ]:
from gensim.models import FastText
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

dataset = load_dataset('large_spanish_corpus', name='DGT')

similar_words = ["Importación", "Europea", "permita", "Madrid", "Bogotá"]

# 300 words
modelW2V300 = Word2Vec(vector_size=300, window=5, min_count=1, workers=4)
modelW2V300.build_vocab(corpus_iterable=sentences[1:300])

for word in similar_words:
  print(word)
  try:
    print(modelW2V300.wv.most_similar(word))
  except:
    print(word + " not in vocabulary")

In [ ]:
# 1M words
modelW2V1M = Word2Vec(vector_size=300, window=5, min_count=1, workers=4)
modelW2V1M.build_vocab(corpus_iterable=sentences[1:1000000])

for word in similar_words:
  print(word)
  try:
    print(modelW2V1M.wv.most_similar(word))
  except:
    print(word + " not in vocabulary")

In [ ]:
# all words
modelW2VAll = Word2Vec(vector_size=300, window=5, min_count=1, workers=4)
modelW2VAll.build_vocab(corpus_iterable=sentences)

for word in similar_words:
  print(word)
  try:
    print(modelW2VAll.wv.most_similar(word))
  except:
    print(word + " not in vocabulary")